In [103]:
#%%
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from PIL import Image


# Loading all relevant datafiles:

dictionary (as pickle - deserializing), csv, meta (json)

In [134]:
Exp_2_pckl={}
path = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2/'
with open(path + 'Exp_design_2_ID_4.pkl', 'rb') as f:
    Exp_2_pckl = pickle.load(f)

Exp_2_csv = pd.read_csv(path +'/Exp_design_2_4.csv', index_col=None) #length same as number of keys in pckl; group by measurement_id and well_id -> to group also the identical spheroids in pckl
Exp_2_json = pd.read_json('C:/Users/srboval1/InstruProject/NikonPipes/dataStore/metalib.json') 


## Keys of nested dictionaries


scripts for understanding the structure

In [152]:
print('Pckl: Number of individual spheroids (- each will have multiple timestamps as distinct masks):',len(Exp_2_pckl.keys()))
print(f"Csv length:{(Exp_2_csv.shape)}")
print(f"Pickle number of main keys: {len(Exp_2_pckl.keys())}")
print(f"Csv columns: {Exp_2_csv.columns.to_list()}")
print("")
print("Checking keys-dates from json:")
print(Exp_2_json.get(key="240521").get("m"))

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

print(search_json(240521))


Pckl: Number of individual spheroids (- each will have multiple timestamps as distinct masks): 168
Csv length:(168, 8)
Pickle number of main keys: 168
Csv columns: ['Unnamed: 0', 'seeding_density', 'cell_label', 'well_id', 'measurement_id', 'matrix', 'day', 'ID_running']

Checking keys-dates from json:
0.660045336571487
0.660045336571487


Cleaning, ordering columns, grouping

In [153]:
Exp_2_csv = Exp_2_csv.drop(columns=(['Unnamed: 0']))

order_csv = ['ID_running','day', 'matrix', 'cell_label','seeding_density','measurement_id','well_id']  
Exp_2_csv = Exp_2_csv[order_csv]

print(Exp_2_csv.head())
print('Number of groups after measurement_id and well_id grouping:',(Exp_2_csv.groupby(['measurement_id', 'well_id'])).ngroups)

#avoid pandas conflict of copy/view
Exp_2_csv= Exp_2_csv.copy()

#setting id for each distinct spheroid - grouped by all parameters that must match for the same spheroid; shall be same as above
Exp_2_csv['spheroid_id'] = Exp_2_csv.groupby(['measurement_id', 'well_id','cell_label','matrix']).ngroup()
print(f"Number of distinct spheroids: {len(Exp_2_csv['spheroid_id'].unique())}")

   ID_running     day    matrix cell_label  seeding_density  measurement_id  \
0           0  240226  collagen   DCIS.COM              500               0   
1           1  240301  collagen     MCF10A              500               1   
2           2  240301  collagen     MCF10A              500               1   
3           3  240301  collagen     MCF10A              500               1   
4           4  240301  collagen     MCF10A              500               1   

   well_id  
0        6  
1        0  
2        2  
3        3  
4        4  
Number of groups after measurement_id and well_id grouping: 98
Number of distinct spheroids: 98


In [154]:
print(Exp_2_json)

                name                                             240520  \
coords            []  [[[1090, 475], [1714, 1316]], [[939, 812], [17...   
matrix            []                                             IPN3mM   
cell              []  [T, T, DCIS.COM, DCIS.COM, DCIS.COM, MCF10A, M...   
n_cells           []                                                  1   
ignore            []                                                [6]   
stain             []                                                NaN   
dim                0                                                 3D   
incubation_time    0                                                 48   
seeding_density    0                [500, 500, 500, 500, 500, 500, 500]   
passage_number     0                                                NaN   
other              0                                              other   
dt               NaN                                             1800.0   
n_fields         NaN     

In [155]:

concentration_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
material_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
reverse_material_mapping = {
'2mgml':'collagen 2mg/ml',
'3mM':'IPN 2.5mM',
'40mM':'IPN 40mM'
}
cell_mapping = {
'MCF10A': '1',
'DCIS.COM': '2',
'MCF10A.DCIS.COM': '2',
'MCF10ADCIS.com': '2',
'MCF10AT': '3'
}


Checking pixel size converstions to match

In [10]:
pixel_size = 0.32*1e-6

print('List of contour coordinates:',Exp_2_pckl[2]["masks"][10])
print('Linda calculated area [pixels]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10]))
print('re-calculated area in [m]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10])*(0.660535171355723**2))

print('Arttu calculated area:',Exp_2_pckl[2]["area"][10])

List of contour coordinates: [[[1106 1193]]

 [[1105 1194]]

 [[1104 1194]]

 ...

 [[1109 1193]]

 [[1108 1193]]

 [[1107 1193]]]
Linda calculated area [pixels]: 126034.0
re-calculated area in [m]: 54989.480215568066
Arttu calculated area: 54989.480215568066


In [56]:
parameter= 240521
Exp_2_json.get(key=str(parameter)).get("m")

json_days= Exp_2_json.keys()
csv_days = Exp_2_csv["day"].unique()

 
print('No of unique days in metafile:',len(Exp_2_json.keys()))
print('Unique days in metafile:',(Exp_2_json.keys()))


print('No of unique days in csv:',len(Exp_2_csv["day"].unique()))
print('Unique days in csv:',(Exp_2_csv["day"].unique()))


No of unique days in metafile: 25
Unique days in metafile: Index(['name', '240520', '240521', '240522', '240519', '230417', '240502',
       '240226', '240301', '240302', '240305', '240303', '240311', '240312',
       '230418', '240614', '240307', '240426', '240715', '240716', '240721',
       '240722', '240723', '240724', '240725'],
      dtype='object')
No of unique days in csv: 21
Unique days in csv: [240715 240520 240522 240721 240724 240722 240723 240614 240307 240426
 240519 240521 240226 240301 240302 240303 240305 240311 240312 230417
 230418]


Combining json and csv into metadata

In [176]:
#%%
#pixel_size = 0.32*1e-6
df_collecting = []
df_all = []

def search_csv(key_id):
    well = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'well_id'].iloc[0]
    measurement = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'measurement_id'].iloc[0]
    cell_line = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'cell_label'].iloc[0]
    material = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'matrix'].iloc[0]
    day = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'day'].iloc[0]
    spheroid_id = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'spheroid_id'].iloc[0]
    return well, measurement, cell_line, material, day, spheroid_id

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

df_collecting = []
df_all = []

for key in Exp_2_pckl.keys():
    df_temp = []

    if len(Exp_2_pckl[key]["time"]) == 0:
        print(f"No time data for key: {key}")
        continue  # Skips the next key

    for i in range(len(Exp_2_pckl[key]["time"])):
        mask = Exp_2_pckl[key]["masks"][i]
        time = Exp_2_pckl[key]["time"][i]
        
        well_id, measurement_id, cell, matrix, date,spheroid_id = search_csv(key)
        if any(val is None for val in [well_id, measurement_id, cell, matrix, date,spheroid_id]):
            print(f"Missing data for key: {key}")
            continue  # Skips to next key
        
        pixel_transf = search_json(date)
        if pixel_transf is None:
            print(f"Missing pixel transformation data for date: {date}")
            continue  # Skip if no valid transformation is found
        
        area_pixels2 = cv2.contourArea(mask)
        area_µm = area_pixels2 * (pixel_transf ** 2)
        
        perimeter = cv2.arcLength(mask, closed=True) * pixel_transf
        compactness = 4 * math.pi * area_µm
        rect = cv2.minAreaRect(mask)
        width, height = rect[1]
        aspect_ratio = width / height
        convexhull = cv2.convexHull(mask)
        convexhull_perimeter = cv2.arcLength(convexhull, closed=True) * pixel_transf
        convexhull_area = cv2.contourArea(convexhull) * (pixel_transf ** 2)
        convexity_perimeter = convexhull_perimeter / perimeter
        convexity_area = convexhull_area / area_µm
        
        info = {
            "ID_running": key,
            "spheroid_id": spheroid_id,
            "time_id": i + 1,
            "incubation_time": time,
            "area_pixels**2": area_pixels2,
            "area_µm": area_µm,
            "perimeter_µm": perimeter,
            "compactness": compactness,
            "aspect_ratio": aspect_ratio,
            "convex hull perimeter": convexhull_perimeter,
            "convex hull area": convexhull_area,
            "convexity_perimeter": convexity_perimeter,
            "convexity_area": convexity_area,
            "well_id": well_id,
            "measurement_id": measurement_id,
            "cell_line": cell,
            "material": matrix,
            "date": date
        }
        df_temp.append(pd.DataFrame([info]))

    if df_temp:
        df_collecting.append(pd.concat(df_temp, ignore_index=True))
    else:
        print(f"No data to append for key: {key}")

df_all = pd.concat(df_collecting, ignore_index=True)


No time data for key: 127


In [177]:
df_all.columns

Index(['ID_running', 'spheroid_id', 'time_id', 'incubation_time',
       'area_pixels**2', 'area_µm', 'perimeter_µm', 'compactness',
       'aspect_ratio', 'convex hull perimeter', 'convex hull area',
       'convexity_perimeter', 'convexity_area', 'well_id', 'measurement_id',
       'cell_line', 'material', 'date'],
      dtype='object')

Additional parameters into metadata

In [178]:
df_collecting_normalized = []
df_all_normalized =[]

#Growth column ((size at t-size at zero)/size at zero)
for i in df_all['spheroid_id'].unique(): #looping through spheroids            
    specific_spheroid = df_all[(df_all['spheroid_id'] == i)]
    min_time = np.argmin(specific_spheroid['incubation_time'])

    area_µm0 = specific_spheroid.iloc[min_time]['area_µm']
    compactness0 = specific_spheroid.iloc[min_time]['compactness']
    aspect_ratio0 = specific_spheroid.iloc[min_time]['aspect_ratio']
    convexity_perimeter0 = specific_spheroid.iloc[min_time]['convexity_perimeter']
    convexity_area0 = specific_spheroid.iloc[min_time]['convexity_area']
    
    specific_spheroid.loc[:,'growth_rate_%'] = ((specific_spheroid['area_µm'] - area_µm0 )/ area_µm0)*100
    specific_spheroid.loc[:,'area_µm_N'] = ((specific_spheroid['area_µm'])/ area_µm0)
    specific_spheroid.loc[:,'compactness_N'] = ((specific_spheroid['compactness'])/ compactness0)
    specific_spheroid.loc[:,'aspect_ratio_N'] = ((specific_spheroid['aspect_ratio'] )/ aspect_ratio0)
    specific_spheroid.loc[:,'convexity_perimeter_N'] = ((specific_spheroid['convexity_perimeter'])/ convexity_perimeter0)
    specific_spheroid.loc[:,'convexity_area_N'] = ((specific_spheroid['convexity_area'])/ convexity_area0)

    df_collecting_normalized.append(pd.concat([specific_spheroid]))

df_all_normalized = pd.concat(df_collecting_normalized, ignore_index=True)

saving_path= 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2'
df_all_normalized.to_csv(saving_path+'/Exp2_data.csv')  

C:\Users\srboval1\AppData\Local\Temp\ipykernel_10948\1690665758.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_spheroid.loc[:,'growth_rate_%'] = ((specific_spheroid['area_µm'] - area_µm0 )/ area_µm0)*100
C:\Users\srboval1\AppData\Local\Temp\ipykernel_10948\1690665758.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_spheroid.loc[:,'area_µm_N'] = ((specific_spheroid['area_µm'])/ area_µm0)
C:\Users\srboval1\AppData\Local\Temp\ipykernel_10948\1690665758.py:17: SettingWithCopyWarning:

In [179]:
df_all_normalized.columns

Index(['ID_running', 'spheroid_id', 'time_id', 'incubation_time',
       'area_pixels**2', 'area_µm', 'perimeter_µm', 'compactness',
       'aspect_ratio', 'convex hull perimeter', 'convex hull area',
       'convexity_perimeter', 'convexity_area', 'well_id', 'measurement_id',
       'cell_line', 'material', 'date', 'growth_rate_%', 'area_µm_N',
       'compactness_N', 'aspect_ratio_N', 'convexity_perimeter_N',
       'convexity_area_N'],
      dtype='object')

Visualizing the mask

In [38]:
#%%
save = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/TimeLapse/'
width, height = 2304, 2304

# Create a blank white image using Pillow and converting it into array


for key in Exp_2_pckl.keys() :
    time = Exp_2_pckl[key]["time"]
    
    blank_image = Image.new('RGB', (width, height), color = (255, 255, 255))
    blank_image_array = np.asarray(blank_image)

    new_image=np.zeros((height, width, 3), np.uint8)


    for i in range(len(Exp_2_pckl[key]["time"])): #looping through time
        mask = Exp_2_pckl[key]["masks"][i]
        spheroid_id = key+1
        blank_image_array= cv2.drawContours(blank_image_array, Exp_2_pckl[key]["masks"][i], -1, (0, 0, 0), 1)
        #print( f'{unique_material}- {unique_line}; min: {min_value} and max {max_value}')

    plt.title(f'{matrix} - {cell_line}; spheroid id: {spheroid_id}') 
    plt.imshow(new_image)
    filename = f'{cell_line}_{matrix}_id_{id}.tiff'

    cv2.imwrite(save+filename, blank_image_array) #saving the contours

NameError: name 'cell_line' is not defined